# Titanic Kaggle By Wesley Mulia Salim

## Import Packages

In [99]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.impute import SimpleImputer
from lightgbm import LGBMClassifier

## Import Data

In [64]:
train_data = pd.read_csv("train.csv")
train_data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [65]:
test_data = pd.read_csv("test.csv")
test_data

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


## Preprocess the data

In [66]:
train_data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [67]:
test_data.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [68]:
train_data.drop(["Name","Ticket","Cabin"], axis = 1, inplace = True)
test_data.drop(["Name","Ticket","Cabin"], axis = 1, inplace = True)

In [69]:
train_data

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,male,22.0,1,0,7.2500,S
1,2,1,1,female,38.0,1,0,71.2833,C
2,3,1,3,female,26.0,0,0,7.9250,S
3,4,1,1,female,35.0,1,0,53.1000,S
4,5,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...,...
886,887,0,2,male,27.0,0,0,13.0000,S
887,888,1,1,female,19.0,0,0,30.0000,S
888,889,0,3,female,NaN,1,2,23.4500,S
889,890,1,1,male,26.0,0,0,30.0000,C


In [70]:
test_data

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,male,34.5,0,0,7.8292,Q
1,893,3,female,47.0,1,0,7.0000,S
2,894,2,male,62.0,0,0,9.6875,Q
3,895,3,male,27.0,0,0,8.6625,S
4,896,3,female,22.0,1,1,12.2875,S
...,...,...,...,...,...,...,...,...
413,1305,3,male,NaN,0,0,8.0500,S
414,1306,1,female,39.0,0,0,108.9000,C
415,1307,3,male,38.5,0,0,7.2500,S
416,1308,3,male,NaN,0,0,8.0500,S


In [71]:
test_data["Fare"] = test_data["Fare"].fillna(test_data["Fare"].median())

In [72]:
fare_category = []
fare = train_data['Fare']
for f in fare:
    if f<100:
        fare_category.append('Low')
    elif f<250:
        fare_category.append('Average')
    else:
        fare_category.append('High')
train_data['Fare Category'] = fare_category

In [73]:
fare_category = []
fare = test_data['Fare']
for f in fare:
    if f<100:
        fare_category.append('Low')
    elif f<250:
        fare_category.append('Average')
    else:
        fare_category.append('High')
test_data['Fare Category'] = fare_category

In [74]:
train_data[['Embarked']] = pd.DataFrame(SimpleImputer(strategy='most_frequent').fit_transform(train_data[['Embarked']]))
train_data[['Age']] = pd.DataFrame(SimpleImputer(strategy='mean').fit_transform(train_data[['Age']]))

In [75]:
test_data[['Age']] = pd.DataFrame(SimpleImputer(strategy='mean').fit_transform(test_data[['Age']]))

In [76]:
train_data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Sex              0
Age              0
SibSp            0
Parch            0
Fare             0
Embarked         0
Fare Category    0
dtype: int64

In [77]:
test_data.isnull().sum()

PassengerId      0
Pclass           0
Sex              0
Age              0
SibSp            0
Parch            0
Fare             0
Embarked         0
Fare Category    0
dtype: int64

In [78]:
train_data.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Fare Category
0,1,0,3,male,22.0,1,0,7.2500,S,Low
1,2,1,1,female,38.0,1,0,71.2833,C,Low
2,3,1,3,female,26.0,0,0,7.9250,S,Low
3,4,1,1,female,35.0,1,0,53.1000,S,Low
4,5,0,3,male,35.0,0,0,8.0500,S,Low


In [79]:
age_group = []
age = train_data['Age']
for i in age:
    if i<13:
        age_group.append('Children')
    elif i<20:
        age_group.append('Teenagers')
    elif i<40:
        age_group.append('Adults')
    elif i<60:
        age_group.append('Middle Age')
    else:
        age_group.append('Senior Citizen')

train_data['Age Category'] = age_group

In [80]:
age_group = []
age = test_data['Age']
for i in age:
    if i<13:
        age_group.append('Children')
    elif i<20:
        age_group.append('Teenagers')
    elif i<40:
        age_group.append('Adults')
    elif i<60:
        age_group.append('Middle Age')
    else:
        age_group.append('Senior Citizen')

test_data['Age Category'] = age_group

In [81]:
y_train = train_data["Survived"]

In [82]:
features = ["Pclass", "Sex", "SibSp", "Parch", "Fare Category", "Embarked", "Age Category"]
X_train = pd.get_dummies(train_data[features], drop_first = True)
X_test = pd.get_dummies(test_data[features], drop_first = True)

In [83]:
X_test

,Pclass,SibSp,Parch,Sex_male,Fare Category_High,Fare Category_Low,Embarked_Q,Embarked_S,Age Category_Children,Age Category_Middle Age,Age Category_Senior Citizen,Age Category_Teenagers
0,3,0,0,1,0,1,1,0,0,0,0,0
1,3,1,0,0,0,1,0,1,0,1,0,0
2,2,0,0,1,0,1,1,0,0,0,1,0
3,3,0,0,1,0,1,0,1,0,0,0,0
4,3,1,1,0,0,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
413,3,0,0,1,0,1,0,1,0,0,0,0
414,1,0,0,0,0,0,0,0,0,0,0,0
415,3,0,0,1,0,1,0,1,0,0,0,0
416,3,0,0,1,0,1,0,1,0,0,0,0


## Model Testing

### Decision Tree Classification

#### Feature Scaling

In [104]:
sc = StandardScaler()
X_train_sc = sc.fit_transform(X_train)
X_test_sc = sc.transform(X_test)

In [19]:
X_train_sc

array([[ 0.82520863,  0.43135024, -0.47432585, ...,  0.73534203,
        -0.30794088,  0.61679395],
       [-1.57221121,  0.43135024, -0.47432585, ..., -1.35991138,
        -0.30794088, -1.62128697],
       [ 0.82520863, -0.47519908, -0.47432585, ..., -1.35991138,
        -0.30794088,  0.61679395],
       ...,
       [ 0.82520863,  0.43135024,  2.00611934, ..., -1.35991138,
        -0.30794088,  0.61679395],
       [-1.57221121, -0.47519908, -0.47432585, ...,  0.73534203,
        -0.30794088, -1.62128697],
       [ 0.82520863, -0.47519908, -0.47432585, ...,  0.73534203,
         3.24737656, -1.62128697]])

In [20]:
classifierDT = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifierDT.fit(X_train_sc, y_train)

DecisionTreeClassifier(criterion='entropy', random_state=0)

In [21]:
# predictions = classifierDT.predict(X_test)

In [22]:
# output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
# output.to_csv('submissionDT.csv', index=False)
# print("Your submission was successfully saved!")

### SVM

In [31]:
classifierSVC = SVC(kernel = 'linear', probability = True, random_state = 0)
classifierSVC.fit(X_train_sc, y_train)

SVC(kernel='linear', probability=True, random_state=0)

In [32]:
predictions = classifierSVC.predict(X_test)

C:\Users\Wesley\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but SVC was fitted without feature names
  warnings.warn(


In [33]:
#output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
#output.to_csv('submissionSVC.csv', index=False)
#print("Your submission was successfully saved!")

Your submission was successfully saved!


### LightGBM

In [100]:
classifierLGB = LGBMClassifier(learning_rate=0.02, n_estimators=300)
classifierLGB.fit(X_train, y_train)

LGBMClassifier(learning_rate=0.02, n_estimators=300)

In [101]:
predictions = classifierLGB.predict(X_test)

In [102]:
# output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
# output.to_csv('submissionLGB.csv', index=False)
# print("Your submission was successfully saved!")

Your submission was successfully saved!


### Random Forest

In [105]:
classifierRF = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifierRF.fit(X_train_sc, y_train)

RandomForestClassifier(criterion='entropy', n_estimators=10, random_state=0)

In [107]:
predictions = classifierRF.predict(X_test_sc)

In [108]:
# output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
# output.to_csv('submissionRF.csv', index=False)
# print("Your submission was successfully saved!")

Your submission was successfully saved!


### XGBoost

In [91]:
classifierXG = XGBClassifier(learning_rate= 0.5, n_estimators= 10)
classifierXG.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.5, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=10, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [92]:
predictions = classifierXG.predict(X_test)

In [94]:
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('submissionXG.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!


#### Through 12 submissions, I manage to get my best accuracy prediction with the XGBoost model, with a score of 0.78947 on the Kaggle Website.